In [1]:
# Install libraries
!pip install python-dotenv
!pip install pymongo[srv]
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Import necessary modules
from dotenv import load_dotenv
from datetime import datetime
from pprint import pprint
import pandas as pd
import os
import bson
import pymongo

load_dotenv()

True

In [3]:
print(pymongo.__version__)

4.3.3


In [4]:
# Connect to mongoDB

client = pymongo.MongoClient(os.environ.get("CONN_STRING"))

In [5]:
print(client.primary)

('ac-tzhuuxz-shard-00-01.edbfm1z.mongodb.net', 27017)


In [6]:
print(client.secondaries)

{('ac-tzhuuxz-shard-00-02.edbfm1z.mongodb.net', 27017), ('ac-tzhuuxz-shard-00-00.edbfm1z.mongodb.net', 27017)}


In [7]:
client.list_database_names()

['ETA_Trucks', 'admin', 'local']

In [5]:
# Connect to Database

db = client['ETA_Trucks']

In [9]:
db.list_collection_names()

['truck_schedule_details',
 'truck_details',
 'driver_details',
 'route_details',
 'city_details']

In [10]:
# Find a single document in the "city_details" collection where the "name" is equal to "Alabama"
filter = {'name': {"$eq": "Alabama"}}

# Find and print the matching document
db.city_details.find_one(filter)


{'_id': ObjectId('63cf28c750cce1d3ca60f9f9'),
 'city_id': 1,
 'name': 'Alabama',
 'lat': 42.998,
 'lon': -78.185}

In [11]:
# Find a document in the "city_details" collection using the specified ObjectId
filter = {"_id": bson.ObjectId('63cf28c750cce1d3ca60f9fe')}

# Find and print the matching document
db.city_details.find_one(filter)


{'_id': ObjectId('63cf28c750cce1d3ca60f9fe'),
 'city_id': 6,
 'name': 'Colorado Springs',
 'lat': 38.834,
 'lon': -104.821}

In [13]:
# Find the most efficient diesel trucks with age greater than 12
filter = {
    "fuel_type": 'diesel',
    "truck_age": {"$gt": 12}
}

# Define projection to exclude '_id' and 'fuel_type' from the result
projection = {"_id": 0, "fuel_type": 0}

# Define sorting sequence for the result
sort_seq = [
    ("truck_age", pymongo.ASCENDING),
    ("load_capacity_pounds", pymongo.DESCENDING),
    ("mileage_mpg", pymongo.DESCENDING)
]

# Find and print the top 5 matching documents, sorted accordingly
result = db.truck_details.find(filter, projection).sort(sort_seq).limit(5)

for i in result:
    pprint(i)
    print("\n")


{'load_capacity_pounds': 20000,
 'mileage_mpg': 29,
 'truck_age': 13,
 'truck_id': '7590026a0d2a4c1d8bfe7426765c260b'}


{'load_capacity_pounds': 20000,
 'mileage_mpg': 28,
 'truck_age': 13,
 'truck_id': '4d8ed7b8fab346b1b057a21f332a343e'}


{'load_capacity_pounds': 20000,
 'mileage_mpg': 28,
 'truck_age': 13,
 'truck_id': '55c5b1edae2c47e6a4e4bade3835f610'}


{'load_capacity_pounds': 20000,
 'mileage_mpg': 28,
 'truck_age': 13,
 'truck_id': '0ad2682b55b84d91868d9528226f3aaf'}


{'load_capacity_pounds': 20000,
 'mileage_mpg': 28,
 'truck_age': 13,
 'truck_id': '7b74e1f38e694880b4723fe38b396fb0'}


CPU times: user 6.54 ms, sys: 1.26 ms, total: 7.8 ms
Wall time: 394 ms


In [23]:
# Count the number of documents for young, proactive, male drivers reliable for long distances
filter = {
    "$and": [
        {"gender": "male"},
        {"experience": {"$gt": 10}},
        {"driving_style": {"$in": ["proactive"]}}
    ],
    "$or": [
        {"$and": [
            {"age": {"$gte": 35}},
            {"age": {"$lte": 45}}
        ]},
        {"ratings": {"$gt": 4}}
    ]
}

# Print the count of matching documents
print(db.driver_details.count_documents(filter))

# Define sorting sequence for the result
sort_seq = [("ratings", -1), ("age", 1)]

# Find and print the top 5 matching documents, sorted accordingly
result = db.driver_details.find(filter).sort(sort_seq).limit(5)

for i in result:
    pprint(i)
    print("\n")


20389
{'_id': ObjectId('63cf2c1050cce1d3ca66fae2'),
 'age': 39,
 'driver_id': '76a283a440aa4799a87086b24a65f1db',
 'driving_style': 'proactive',
 'experience': 11,
 'gender': 'male',
 'name': 'Michael Mcconnell',
 'ratings': 5,
 'vehicle_no': 'a7022bf811ed40ef911d775c9da1ca9e'}


{'_id': ObjectId('63cf2c0650cce1d3ca66c547'),
 'age': 40,
 'driver_id': '046ad46921104e6db54d1a2399a7944f',
 'driving_style': 'proactive',
 'experience': 11,
 'gender': 'male',
 'name': 'Andrew Forbes',
 'ratings': 5,
 'vehicle_no': '254869b1ffac41feae6790037ed12844'}


{'_id': ObjectId('63cf2bec50cce1d3ca662b18'),
 'age': 40,
 'driver_id': '843f66e8bcf2479e9997a50603cb7a97',
 'driving_style': 'proactive',
 'experience': 11,
 'gender': 'male',
 'name': 'Scott Johnson',
 'ratings': 5,
 'vehicle_no': '1efbcb288cb843c4bc0ff7b941ca5b70'}


{'_id': ObjectId('63cf2b8550cce1d3ca64148c'),
 'age': 41,
 'driver_id': '4780d7706490482ca19e4f5ed1dbba81',
 'driving_style': 'proactive',
 'experience': 22,
 'gender': 'male',


In [24]:
# Find documents in the "driver_details" collection where the first name starts with "Tyler" using regex
filter = {"name": {"$regex": "^Tyler .*", "$options": 'ms'}}

# Limit the result to 2 documents
result = db.driver_details.find(filter).limit(2)

# 'm' for caret(^) - startWith, $ - endWith
# 's' for period(.) - all characters

# Print the matching documents
for i in result:
    pprint(i)


{'_id': ObjectId('63cf2b8450cce1d3ca6410a1'),
 'age': 41,
 'driver_id': '40287c65b040442f899c0affbcad207a',
 'driving_style': 'proactive',
 'experience': 22,
 'gender': 'male',
 'name': 'Tyler Saunders',
 'ratings': 3.3,
 'vehicle_no': '765d4022c30848c3a4517f003d2689e7'}
{'_id': ObjectId('63cf2b8450cce1d3ca6410ff'),
 'age': 51,
 'driver_id': 'd0f65590f84c42a697db283a6795666c',
 'driving_style': 'proactive',
 'experience': 12,
 'gender': 'male',
 'name': 'Tyler Johnson',
 'ratings': 2.9,
 'vehicle_no': 'f1080c451a90403c97e2db8699c26c3f'}


In [25]:
# Find documents in the "driver_details" collection where the last name ends with "Dean" using regex
filter = {"name": {"$regex": ".* Dean$", "$options": 'ms'}}

# Limit the result to 2 documents
result = db.driver_details.find(filter).limit(2)

# Print the matching documents
for i in result:
    pprint(i)


{'_id': ObjectId('63cf2b8450cce1d3ca64138c'),
 'age': 44,
 'driver_id': 'e7bb0b7942994e7d8eed9a8639e63713',
 'driving_style': 'conservative',
 'experience': 14,
 'gender': 'male',
 'name': 'Tyler Dean',
 'ratings': 4.4,
 'vehicle_no': 'ccd30d82a450492884cd0ce67814ea49'}
{'_id': ObjectId('63cf2b8550cce1d3ca641704'),
 'age': 46,
 'driver_id': '7bef3146c7ee4a7593e013334e0d2cd2',
 'driving_style': 'proactive',
 'experience': 14,
 'gender': 'male',
 'name': 'Robert Dean',
 'ratings': 1.5,
 'vehicle_no': 'a3604250716849caad7fb71f40872d59'}


In [26]:
# Update a document in the "truck_details" collection by setting the "fines" field to a new list
filter = {"truck_id": "2f0705c938e545a0aa03c505ec97751d"}

update = {
    "$set": {
        "fines": [
            {
                "date": datetime.utcnow(),
                "amount": 500,
                "reason": "Speed Limit",
                "driver_id": "c3173876aeac4604863da23d0a90f6f3"
            }
        ]
    }
}

# Update a single document using update_one
db.truck_details.update_one(filter, update)


In [27]:
# Update a document in the "truck_details" collection by pushing a new element to the "fines" list
filter = {"truck_id": "2f0705c938e545a0aa03c505ec97751d"}

update = {
    "$push": {
        "fines": {
            "date": datetime.utcnow(),
            "amount": 5000,
            "reason": "Red Light Crossed",
            "driver_id": "c3173876aeac4604863da23d0a90f6f3"
        }
    }
}

# Update a single document using update_one
db.truck_details.update_one(filter, update)


In [28]:
# Update all documents in the "truck_details" collection
# Increment the "truck_age" field by 1 for each document
filter = {}

update = {
    "$inc": {
        "truck_age": 1
    }
}

# Update multiple documents using update_many
db.truck_details.update_many(filter, update)


In [29]:
# Update documents in the "truck_details" collection where the "fuel_type" is "gas"
# Decrement the "mileage_mpg" field by 2 for each matching document
filter = {"fuel_type": "gas"}

update = {
    "$inc": {
        "mileage_mpg": -2
    }
}

# Update multiple documents using update_many
db.truck_details.update_many(filter, update)


In [15]:
# Upsert (update or insert) a document in the "city_details" collection with the specified filter
# If a document with "name" equal to "Houston" exists, update its fields; otherwise, insert a new document
filter = {"name": "Houston"}

update = {
    "$set": {
        "name": "Houston",
        "city_id": 50,
        "lat": 29.74,
        "lon": -95.46
    }
}

# Use upsert=True to perform an upsert operation
db.city_details.update_many(filter, update, upsert=True)


In [16]:
# Delete documents from the "city_details" collection where the "name" field is "Houston"
db.city_details.delete_many({"name": "Houston"})


In [17]:
# # Get execution details of the query without indexes
pprint(db.truck_schedule_details.find({"origin":"Alabama"}).sort("date",pymongo.DESCENDING).explain())

{'$clusterTime': {'clusterTime': Timestamp(1674618400, 1),
                  'signature': {'hash': b'\xa8\xe0\x08\xac\x9e\xb4\xf0\x87'
                                        b'\xc4\xa0\xfe\x00\x16<\td=f\x8c\x7f',
                                'keyId': 7154063244682854402}},
 'command': {'$db': 'ETA_Trucks',
             'filter': {'origin': 'Alabama'},
             'find': 'truck_schedule_details',
             'sort': {'date': -1}},
 'executionStats': {'allPlansExecution': [],
                    'executionStages': {'advanced': 13620,
                                        'executionTimeMillisEstimate': 51,
                                        'inputStage': {'advanced': 13620,
                                                       'direction': 'forward',
                                                       'docsExamined': 566110,
                                                       'executionTimeMillisEstimate': 50,
                                                       'fi

In [18]:
from pymongo import IndexModel, ASCENDING, DESCENDING

# Define a compound index on "origin" (ASCENDING) and "date" (DESCENDING)
index1 = IndexModel([("origin", ASCENDING), ("date", DESCENDING)])

# Define a single-field index on "destination" (ASCENDING)
index2 = IndexModel([("destination", ASCENDING)])

# Create the defined indexes on the "truck_schedule_details" collection
db.truck_schedule_details.create_indexes([index1, index2])


['origin_1_date_-1', 'destination_1']

In [19]:
# Get execution details of the query
pprint(db.truck_schedule_details.find({"origin": "Alabama"}).sort("date", pymongo.DESCENDING).explain())


{'$clusterTime': {'clusterTime': Timestamp(1674619148, 3),
                  'signature': {'hash': b'\x05\xa5\x8f\xe4\x97\x03\x1a\xc7'
                                        b'/\x9a\xd1\x89\xe3aXeC\xb6\xe9~',
                                'keyId': 7154063244682854402}},
 'command': {'$db': 'ETA_Trucks',
             'filter': {'origin': 'Alabama'},
             'find': 'truck_schedule_details',
             'sort': {'date': -1}},
 'executionStats': {'allPlansExecution': [],
                    'executionStages': {'advanced': 13620,
                                        'alreadyHasObj': 0,
                                        'docsExamined': 13620,
                                        'executionTimeMillisEstimate': 36,
                                        'inputStage': {'advanced': 13620,
                                                       'direction': 'forward',
                                                       'dupsDropped': 0,
                                   

In [20]:
# Retrieve index information for the 'truck_schedule_details' collection
db.truck_schedule_details.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'origin_1_date_-1': {'v': 2, 'key': [('origin', 1), ('date', -1)]},
 'destination_1': {'v': 2, 'key': [('destination', 1)]}}

In [7]:
db2 = client['Ecom']

In [9]:
# Insert documents into 'Quant' collection
db2.Quant.insert_many([
    {"item": "abc", "quantity": 41, "lastModified": datetime.strptime("2023-01-09 01:01:01", "%Y-%m-%d %H:%M:%S")},
    {"item": "def", "quantity": 56, "lastModified": datetime.strptime("2023-01-09 01:01:01", "%Y-%m-%d %H:%M:%S")},
    {"item": "xyz", "quantity": 78, "lastModified": datetime.strptime("2022-02-09 01:01:01", "%Y-%m-%d %H:%M:%S")},
    {"item": "jkl", "quantity": 67, "lastModified": datetime.strptime("2023-01-09 01:01:01", "%Y-%m-%d %H:%M:%S")}
])

# Insert documents into 'sales' collection
db2.sales.insert_many([
    {"_id": 1, "item": "abc", "price": 8, "quantity": 4, "date": datetime.strptime("2022-01-01 01:01:01", "%Y-%m-%d %H:%M:%S")},
    {"_id": 2, "item": "xyz", "price": 8, "quantity": 4, "date": datetime.strptime("2022-02-01 01:01:01", "%Y-%m-%d %H:%M:%S")},
    {"_id": 3, "item": "jkl", "price": 10, "quantity": 5, "date": datetime.strptime("2023-01-01 01:01:01", "%Y-%m-%d %H:%M:%S")},
    {"_id": 4, "item": "abc", "price": 12, "quantity": 6, "date": datetime.strptime("2023-01-01 01:01:01", "%Y-%m-%d %H:%M:%S")},
    {"_id": 5, "item": "def", "price": 16, "quantity": 8, "date": datetime.strptime("2023-01-01 01:01:01", "%Y-%m-%d %H:%M:%S")},
    {"_id": 6, "item": "jkl", "price": 10, "quantity": 5, "date": datetime.strptime("2023-01-01 01:01:01", "%Y-%m-%d %H:%M:%S")},
    {"_id": 7, "item": "abc", "price": 10, "quantity": 5, "date": datetime.strptime("2023-01-01 01:01:01", "%Y-%m-%d %H:%M:%S")}
])


In [17]:
# Function for handling transactions
def callback(session, sales_id, item, quantity, price, date):

    # Accessing 'sales' and 'quant' collections using the provided session's client
    sales = session.client.Ecom.sales
    quant = session.client.Ecom.Quant

    # Transaction Operations

    # Insert a document into 'sales' collection with the provided data
    sales.insert_one({
        "_id": sales_id,
        "item": item,
        "price": price,
        "quantity": quantity,
        "date": date
    }, session=session)

    # Simulate an intentional error (divide by zero) to demonstrate a rollback
    t = 6 / 0

    # Update 'quant' collection by decrementing the quantity of the specified item
    quant.update_one(
        {"item": item},
        {
            "$inc": {
                "quantity": -abs(quantity)
            },
            "$currentDate": {"lastModified": True}
        },
        session=session
    )


In [18]:
# Start a session using the client
with client.start_session() as session:
  
    # Use a transaction within the session
    session.with_transaction(
        
        # Lambda function as the transaction callback
        lambda a: callback(a, sales_id=10, item='xyz', quantity=10, price=20, date=datetime.now())
    )


In [21]:
# MongoDB aggregation pipeline to join "Quant" with "sales" and merge the result into "Inventory" in the "Ecom" database
stage_1 = {
    "$lookup": {
        "from": "sales",       # Collection to join with
        "localField": "item",  # Field from the input documents
        "foreignField": "item",  # Field from the documents of the "sales" collection
        "as": "sales_list"     # Output array field
    }
}  # Stage 1: Join "Quant" with "sales" based on the "item" field

stage_2 = {"$unset": "sales_list._id"}  # Stage 2: Remove the "_id" field from the "sales_list" array

stage_3 = {
    "$merge": {
        "into": {"db": "Ecom", "coll": "Inventory"}  # Stage 3: Merge the result into the "Inventory" collection in the "Ecom" database
    }
}

# Execute the aggregation pipeline
results = db2.Quant.aggregate([stage_1, stage_2, stage_3])

# Iterate through the aggregation results and print them
for i in results:
    pprint(i)
    print("\n")


In [29]:
# MongoDB query to find documents in the "Inventory" collection based on specified conditions within the "sales_list" array
filter = {
    "sales_list": {
        "$elemMatch": {
            "quantity": {"$ne": 4},  # Match elements within "sales_list" where "quantity" is not equal to 4
            "$and": [
                {"date": {"$gte": datetime(2022, 1, 1)}},  # Match elements within "sales_list" where "date" is greater than or equal to January 1, 2022
                {"date": {"$lte": datetime(2023, 2, 1)}}    # Match elements within "sales_list" where "date" is less than or equal to February 1, 2023
            ]
        }
    }
}

# MongoDB query to find documents in the "Inventory" collection based on the specified filter
result = db2.Inventory.find(filter)

# Iterate through the query results and print each document
for i in result:
    pprint(i)
    print('\n')


{'_id': ObjectId('63d2f5b461859bb164d0db8b'),
 'item': 'abc',
 'lastModified': datetime.datetime(2023, 1, 9, 1, 1, 1),
 'quantity': 41,
 'sales_list': [{'date': datetime.datetime(2022, 1, 1, 1, 1, 1),
                 'item': 'abc',
                 'price': 8,
                 'quantity': 4},
                {'date': datetime.datetime(2023, 1, 1, 1, 1, 1),
                 'item': 'abc',
                 'price': 12,
                 'quantity': 6},
                {'date': datetime.datetime(2023, 1, 1, 1, 1, 1),
                 'item': 'abc',
                 'price': 10,
                 'quantity': 5}]}


{'_id': ObjectId('63d2f5b461859bb164d0db8c'),
 'item': 'def',
 'lastModified': datetime.datetime(2023, 1, 9, 1, 1, 1),
 'quantity': 56,
 'sales_list': [{'date': datetime.datetime(2023, 1, 1, 1, 1, 1),
                 'item': 'def',
                 'price': 16,
                 'quantity': 8}]}


{'_id': ObjectId('63d2f5b461859bb164d0db8d'),
 'item': 'xyz',
 'lastModified': date

In [30]:
# MongoDB query to find documents in the "Inventory" collection based on specified conditions
result = db2.Inventory.find({
    "sales_list.quantity": {"$ne": 4},  # Match documents where "quantity" in "sales_list" is not equal to 4
    "$and": [
        {"sales_list.date": {"$gte": datetime(2022, 1, 1)}},  # Match documents where "date" in "sales_list" is greater than or equal to January 1, 2022
        {"sales_list.date": {"$lte": datetime(2023, 2, 1)}}   # Match documents where "date" in "sales_list" is less than or equal to February 1, 2023
    ]
})

# Iterate through the query results and print each document
for i in result:
    pprint(i)
    print('\n')


{'_id': ObjectId('63d2f5b461859bb164d0db8c'),
 'item': 'def',
 'lastModified': datetime.datetime(2023, 1, 9, 1, 1, 1),
 'quantity': 56,
 'sales_list': [{'date': datetime.datetime(2023, 1, 1, 1, 1, 1),
                 'item': 'def',
                 'price': 16,
                 'quantity': 8}]}


{'_id': ObjectId('63d2f5b461859bb164d0db8e'),
 'item': 'jkl',
 'lastModified': datetime.datetime(2023, 1, 9, 1, 1, 1),
 'quantity': 67,
 'sales_list': [{'date': datetime.datetime(2023, 1, 1, 1, 1, 1),
                 'item': 'jkl',
                 'price': 10,
                 'quantity': 5},
                {'date': datetime.datetime(2023, 1, 1, 1, 1, 1),
                 'item': 'jkl',
                 'price': 10,
                 'quantity': 5}]}




In [31]:
# Connect to MongoDB
client = MongoClient("your_mongodb_connection_string")
db2 = client.your_database_name

# Define the filter to match documents with the specified item
filter = {"item": "xyz"}

# Define the update operations
update = {
    "$inc": {
        "quantity": -7  # Decrement the "quantity" field by 7
    },
    "$push": {
        "sales_list": {  # Push a new document into the "sales_list" array
            "item": "xyz",
            "price": 14,
            "quantity": 7,
            "date": datetime.now()  # Set the "date" field with the current date
        }
    },
    "$currentDate": {"lastModified": True}  # Update the "lastModified" field with the current date
}

# Update documents in the "Inventory" collection based on the filter
result = db2.Inventory.update_many(filter, update)

# Print the number of documents matched and modified
print(f"Matched {result.matched_count} documents and modified {result.modified_count} documents.")


In [8]:
# MongoDB aggregation pipeline to calculate VAT for each item in the sales_list
stage_1 = {
    "$set": {
        "VAT": {
            "$map": {
                "input": "$sales_list",  # Input array to map over
                "as": "grade",  # Variable to represent each element in the array
                "in": {"$multiply": ["$$grade.price", 0.05]}  # Multiply each "price" by 0.05
            }
        }
    }
}  # Stage 1: Create a new field "VAT" by calculating VAT for each item in the "sales_list"

# Execute the aggregation pipeline
result = db2.Inventory.aggregate([stage_1])

# Iterate through the aggregation results and print them
for i in result:
    pprint(i)


{'VAT': [0.4, 0.6000000000000001, 0.5],
 '_id': ObjectId('63d2f5b461859bb164d0db8b'),
 'item': 'abc',
 'lastModified': datetime.datetime(2023, 1, 9, 1, 1, 1),
 'quantity': 41,
 'sales_list': [{'date': datetime.datetime(2022, 1, 1, 1, 1, 1),
                 'item': 'abc',
                 'price': 8,
                 'quantity': 4},
                {'date': datetime.datetime(2023, 1, 1, 1, 1, 1),
                 'item': 'abc',
                 'price': 12,
                 'quantity': 6},
                {'date': datetime.datetime(2023, 1, 1, 1, 1, 1),
                 'item': 'abc',
                 'price': 10,
                 'quantity': 5}]}
{'VAT': [0.8],
 '_id': ObjectId('63d2f5b461859bb164d0db8c'),
 'item': 'def',
 'lastModified': datetime.datetime(2023, 1, 9, 1, 1, 1),
 'quantity': 56,
 'sales_list': [{'date': datetime.datetime(2023, 1, 1, 1, 1, 1),
                 'item': 'def',
                 'price': 16,
                 'quantity': 8}]}
{'VAT': [0.4, 1.0, 1.0, 0.700000

In [16]:
# MongoDB aggregation pipeline to find the average age of all drivers
stage_1 = {
    '$group': {
        '_id': None,  # Group by nothing to calculate the average across all documents
        'avg_Age': {'$avg': '$age'}  # Calculate the average age
    }
}  # Stage 1: Group by nothing and calculate the average age

# Execute the aggregation pipeline
result = db.driver_details.aggregate([stage_1])

# Iterate through the aggregation results and print them
for i in result:
    pprint(i)


In [16]:
# MongoDB aggregation pipeline to find the average age of all drivers by all documents, gender, and driving style
facet_stage = {
    '$facet': {
        'AVG_AGE_ALL': [
            {'$group': {
                '_id': None,
                'avg_Age': {'$avg': '$age'},  # Calculate average age
                'total_count': {'$sum': 1}  # Count the total number of documents
            }}
        ],
        'AVG_AGE_GENDER': [
            {'$group': {
                '_id': '$gender',  # Group by gender
                'avg_Age': {'$avg': '$age'},  # Calculate average age within each gender group
                'total_count': {'$sum': 1}  # Count the total number of documents within each gender group
            }}
        ],
        'AVG_AGE_STYLE': [
            {'$group': {
                '_id': '$driving_style',  # Group by driving style
                'avg_Age': {'$avg': '$age'},  # Calculate average age within each driving style group
                'total_count': {'$sum': 1}  # Count the total number of documents within each driving style group
            }}
        ]
    }
}  # Stage: Use $facet to compute multiple aggregations at once

# Execute the aggregation pipeline
result = db.driver_details.aggregate([facet_stage])

# Iterate through the aggregation results and print them
for i in result:
    pprint(i)


In [32]:
# MongoDB aggregation pipeline to join truck and driver details, calculate total_trucks and total_fines, and save to another collection
stage_1 = {
    "$lookup": {
        "from": "truck_details",  # Collection to join with
        "localField": "vehicle_no",  # Field from the input documents
        "foreignField": "truck_id",  # Field from the documents of the "truck_details" collection
        "as": "trucks_assigned"  # Output array field
    }
}  # Stage 1: Join "driver_details" with "truck_details"

stage_2 = {"$unset": ["vehicle_no", "trucks_assigned._id"]}  # Stage 2: Remove unnecessary fields

stage_3 = {'$addFields': {'t_trucks': {'$size': '$trucks_assigned'}}}  # Stage 3: Calculate total_trucks

stage_4 = {"$unwind": "$trucks_assigned"}  # Stage 4: Unwind the array for further processing

stage_5 = {
    '$addFields': {
        't_fines': {
            '$cond': {
                'else': None,
                'if': {'$and': [{'$isArray': '$trucks_assigned.fines'}]},  # Check if fines array exists
                'then': {'$size': '$trucks_assigned.fines'}  # Calculate total_fines
            }
        }
    }
}  # Stage 5: Calculate total_fines

stage_6 = {'$out': {'coll': 'driver_trucks', 'db': 'ETA_Trucks'}}  # Stage 6: Save the result to "driver_trucks" collection

limit_stage = {"$limit": 5}  # Limit the results for testing purposes

temp_stage = {"$match": {"name": "Scott Hawkins"}}  # Additional match stage for testing purposes

# Execute the aggregation pipeline
result = db.driver_details.aggregate([stage_1, stage_2, stage_3, stage_4, stage_5, limit_stage, stage_6])

# Iterate through the aggregation results and print them
for i in result:
    pprint(i)
    print("\n")


In [37]:
# MongoDB aggregation pipeline to add a GeoJSON field and merge the result back into the collection
stage_1 = {
    '$addFields': {
        'location': {
            'coordinates': ['$lon', '$lat'],  # Create a GeoJSON Point with 'lon' and 'lat' values
            'type': 'Point'
        }
    }
}  # Stage 1: Add a GeoJSON field named 'location'

stage_2 = {
    '$merge': {
        'into': 'city_details',  # Merge the result back into the 'city_details' collection
        'on': '_id',  # Use the '_id' field for matching documents
        'whenMatched': 'replace'  # Replace existing documents when a match is found
    }
}  # Stage 2: Merge the result back into the collection

result = db.city_details.aggregate([stage_1, stage_2])

# Iterate through the aggregation results and print them
for i in result:
    pprint(i)
    print("\n")


In [38]:
from pymongo import IndexModel, ASCENDING, DESCENDING

# Define the 2dsphere index model for the "location" field
index1 = IndexModel([("location", "2dsphere")])

# Create the index in the "city_details" collection
db.city_details.create_indexes([index1])


['location_2dsphere']

In [41]:
# MongoDB aggregation pipeline to find the nearest centers for delivery from a given set of coordinates
result = db.city_details.aggregate([
    {
        '$geoNear': {
            'distanceField': 'dist.calculated',  # Output field that contains the calculated distance
            'distanceMultiplier': 0.001,  # Convert distance to kilometers
            'includeLocs': 'dist.location',  # Include location information in 'dist.location'
            'maxDistance': 800000,  # Maximum distance in meters for the centers to be considered
            'minDistance': 0,  # Minimum distance in meters for the centers to be considered
            'near': {'coordinates': [-97.617134, 30.222296], 'type': 'Point'},  # Specify the reference point
            'spherical': True  # Use spherical geometry for more accurate distance calculations
        }
    }
])

# Iterate through the aggregation results and print them
for i in result:
    pprint(i)
    print("\n")


{'_id': ObjectId('63cf28c750cce1d3ca60fa11'),
 'city_id': 25,
 'dist': {'calculated': 211.57425184978044,
          'location': {'coordinates': [-95.538, 29.618], 'type': 'Point'}},
 'lat': 29.618,
 'location': {'coordinates': [-95.538, 29.618], 'type': 'Point'},
 'lon': -95.538,
 'name': 'Missouri City'}


{'_id': ObjectId('63cf28c750cce1d3ca60fa22'),
 'city_id': 42,
 'dist': {'calculated': 278.27607088631123,
          'location': {'coordinates': [-94.903, 29.384], 'type': 'Point'}},
 'lat': 29.384,
 'location': {'coordinates': [-94.903, 29.384], 'type': 'Point'},
 'lon': -94.903,
 'name': 'Texas City'}


{'_id': ObjectId('63cf28c750cce1d3ca60fa1c'),
 'city_id': 36,
 'dist': {'calculated': 584.0221857132893,
          'location': {'coordinates': [-97.516, 35.468], 'type': 'Point'}},
 'lat': 35.468,
 'location': {'coordinates': [-97.516, 35.468], 'type': 'Point'},
 'lon': -97.516,
 'name': 'Oklahoma City'}


{'_id': ObjectId('63cf28c750cce1d3ca60f9fc'),
 'city_id': 4,
 'dist': {'calcu

In [7]:
# MongoDB aggregation pipeline to count the total number of delayed journeys
stage_1 = {
    '$group': {
        '_id': {
            '$toBool': {
                '$convert': {
                    'input': '$delay',  # Convert the 'delay' field to decimal
                    'to': 'decimal'
                }
            }
        },
        'totalcount': {'$sum': 1}  # Count the occurrences of delayed and not delayed journeys
    }
}  # Stage 1: Group by whether the journey is delayed or not, and count the occurrences

stage_2 = {
    '$project': {
        '_id': 0,
        'is_delay': '$_id',  # Create a new field 'is_delay' with the values from '_id'
        'totalcount': '$totalcount'  # Project the total count for each category
    }
}  # Stage 2: Project the required fields for the final result

result = db.truck_schedule_details.aggregate([stage_1, stage_2])

# Iterate through the aggregation results and print them
for i in result:
    pprint(i)
    print("\n")


{'is_delay': True, 'totalcount': 37113}


{'is_delay': False, 'totalcount': 528997}




In [12]:
# MongoDB aggregation pipeline to find the most congested routes
stage_1 = {'$match': {'delay': '1'}}  # Stage 1: Match documents where the delay is equal to "1"

stage_2 = {
    '$group': {
        '_id': {
            'destination': '$destination',  # Group by the 'destination' field
            'origin': '$origin'  # Group by the 'origin' field
        },
        'route_id': {'$first': '$route_id'},  # Take the first 'route_id' encountered in the group
        'totalcount': {'$sum': 1}  # Count the occurrences of delays for each route
    }
}  # Stage 2: Group by destination and origin, aggregate unique route_ids and total delays

stage_3 = {
    '$project': {
        '_id': 0,
        'route': '$_id',  # Create a new field 'route' with the values from '_id'
        'route_id': '$route_id',  # Project the 'route_id' field
        'total_delays': '$totalcount'  # Project the total delays for each route
    }
}  # Stage 3: Project the required fields for the final result

stage_4 = {'$sort': {'total_delays': -1}}  # Stage 4: Sort the results based on total delays in descending order

limit_stage = {"$limit": 10}  # Limit the results to the top 10 congested routes

result = db.truck_schedule_details.aggregate([stage_1, stage_2, stage_3, stage_4, limit_stage])

# Iterate through the aggregation results and print them
for i in result:
    pprint(i)
    print("\n")


{'route': {'destination': 'Pennsylvania Furnace',
           'origin': 'South Dakota Park'},
 'route_id': 'South_Dakota_Park_Pennsylvania_Furnace_97381',
 'total_delays': 104}


{'route': {'destination': 'South Dakota Park',
           'origin': 'Pennsylvania Furnace'},
 'route_id': 'Pennsylvania_Furnace_South_Dakota_Park_1751',
 'total_delays': 104}


{'route': {'destination': 'Nebraska City', 'origin': 'Pennsylvania Furnace'},
 'route_id': 'Pennsylvania_Furnace_Nebraska_City_1193',
 'total_delays': 99}


{'route': {'destination': 'Pennsylvania Furnace', 'origin': 'Nebraska City'},
 'route_id': 'Nebraska_City_Pennsylvania_Furnace_79948',
 'total_delays': 99}


{'route': {'destination': 'Nevada', 'origin': 'Worcester'},
 'route_id': 'Worcester_Nevada_86544',
 'total_delays': 95}


{'route': {'destination': 'Worcester', 'origin': 'Nevada'},
 'route_id': 'Nevada_Worcester_59645',
 'total_delays': 95}


{'route': {'destination': 'Pennsylvania Furnace', 'origin': 'Oklahoma City'},
 'route_

In [14]:
# MongoDB aggregation pipeline to find months with the most delays and total routes affected for the past two years
stage_1 = {'$match': {'delay': '1'}}  # Stage 1: Match documents where the delay is equal to "1"

stage_2 = {
    '$group': {
        '_id': {
            'month': {'$month': '$date'},  # Extract the month from the 'date' field
            'year': {'$year': '$date'}  # Extract the year from the 'date' field
        },
        'routes': {'$addToSet': '$route_id'},  # Add unique route_ids to the 'routes' array
        'total_delays': {'$sum': 1}  # Count the occurrences of delays
    }
}  # Stage 2: Group by month and year, aggregate unique routes and total delays

stage_3 = {
    '$project': {
        '_id': 0,
        'month_year': '$_id',  # Create a new field 'month_year' with the values from '_id'
        'total_delays': '$total_delays',  # Project the 'total_delays' field
        'unique_routes_affected': {'$size': '$routes'}  # Project the size of the 'routes' array
    }
}  # Stage 3: Project the required fields for the final result

stage_4 = {'$sort': {'total_delays': -1}}  # Stage 4: Sort the results based on total delays in descending order

result = db.truck_schedule_details.aggregate([stage_1, stage_2, stage_3, stage_4])

# Iterate through the aggregation results and print them
for i in result:
    pprint(i)
    print("\n")


{'month_year': {'month': 1, 'year': 2017},
 'total_delays': 3740,
 'unique_routes_affected': 1404}


{'month_year': {'month': 3, 'year': 2017},
 'total_delays': 2495,
 'unique_routes_affected': 1208}


{'month_year': {'month': 12, 'year': 2017},
 'total_delays': 2339,
 'unique_routes_affected': 1172}


{'month_year': {'month': 1, 'year': 2018},
 'total_delays': 1826,
 'unique_routes_affected': 1009}


{'month_year': {'month': 3, 'year': 2018},
 'total_delays': 1818,
 'unique_routes_affected': 1042}


{'month_year': {'month': 2, 'year': 2018},
 'total_delays': 1797,
 'unique_routes_affected': 1025}


{'month_year': {'month': 11, 'year': 2018},
 'total_delays': 1735,
 'unique_routes_affected': 975}


{'month_year': {'month': 9, 'year': 2018},
 'total_delays': 1708,
 'unique_routes_affected': 975}


{'month_year': {'month': 4, 'year': 2018},
 'total_delays': 1677,
 'unique_routes_affected': 1062}


{'month_year': {'month': 12, 'year': 2018},
 'total_delays': 1600,
 'unique_routes_affected

In [17]:
# Import the necessary library for pretty printing
from pprint import pprint

# MongoDB aggregation pipeline to retrieve and process truck schedule details
result = db.truck_schedule_details.aggregate([
    {
        "$match": {
            "delay": "1"  # Match documents where the delay is equal to "1"
        }
    },
    {
        "$group": {
            "_id": {
                "year": {"$year": "$date"},  # Extract the year from the 'date' field
                "month": {"$month": "$date"},  # Extract the month from the 'date' field
                "route": "$route_id"  # Group by the 'route_id'
            },
            "total_delays_route_wise": {"$count": {}}  # Count the occurrences of delays for each route
        }
    },
    {
        "$group": {
            "_id": {
                "year": "$_id.year",
                "month": "$_id.month"
            },
            "total_delays": {"$sum": "$total_delays_route_wise"},  # Sum the delays for each month and year
            "unique_routes_affected": {"$count": {}}  # Count the number of unique routes affected
        }
    },
    {
        "$sort": {"total_delays": -1}  # Sort the results based on total delays in descending order
    }
])

# Iterate through the aggregation results and print them
for i in result:
    pprint(i)
    print("\n")


{'_id': {'month': 1, 'year': 2017},
 'total_delays': 3740,
 'unique_routes_affected': 1404}


{'_id': {'month': 3, 'year': 2017},
 'total_delays': 2495,
 'unique_routes_affected': 1208}


{'_id': {'month': 12, 'year': 2017},
 'total_delays': 2339,
 'unique_routes_affected': 1172}


{'_id': {'month': 1, 'year': 2018},
 'total_delays': 1826,
 'unique_routes_affected': 1009}


{'_id': {'month': 3, 'year': 2018},
 'total_delays': 1818,
 'unique_routes_affected': 1042}


{'_id': {'month': 2, 'year': 2018},
 'total_delays': 1797,
 'unique_routes_affected': 1025}


{'_id': {'month': 11, 'year': 2018},
 'total_delays': 1735,
 'unique_routes_affected': 975}


{'_id': {'month': 9, 'year': 2018},
 'total_delays': 1708,
 'unique_routes_affected': 975}


{'_id': {'month': 4, 'year': 2018},
 'total_delays': 1677,
 'unique_routes_affected': 1062}


{'_id': {'month': 12, 'year': 2018},
 'total_delays': 1600,
 'unique_routes_affected': 933}


{'_id': {'month': 5, 'year': 2017},
 'total_delays': 1571,
 